In [ ]:
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dir_chest_xray = os.listdir('/content/drive/MyDrive/Colab Notebooks/datasets/pneumonia')
Nor_Pneumonia = ['normal/','opacity/']

In [ ]:
dir_chest_xray

In [ ]:
dir_chest_xray

In [ ]:
general_path = '/content/drive/MyDrive/Colab Notebooks/datasets/pneumonia/'

In [ ]:
test_path_normal = general_path +  '/' + dir_chest_xray[2] + '/' + Nor_Pneumonia[0]
test_path_affected = general_path + '/' +  dir_chest_xray[2] + '/' + Nor_Pneumonia[1]
train_path_normal = general_path + '/' +  dir_chest_xray[0] + '/' + Nor_Pneumonia[0]
train_path_affected = general_path + '/' + dir_chest_xray[0] + '/' + Nor_Pneumonia[1]
val_path_normal = general_path + '/' + dir_chest_xray[1] + '/' + Nor_Pneumonia[0]
val_path_affected = general_path + '/' +  dir_chest_xray[1] + '/' + Nor_Pneumonia[1]

In [ ]:
Normal_img_path = test_path_normal + '/IM-0001-0001.jpeg'

In [ ]:
train_path = general_path + dir_chest_xray[0] + '/'
test_path = general_path + dir_chest_xray[2] + '/'
val_path = general_path + dir_chest_xray[1] + '/'

In [ ]:
img_Normal = plt.imread(Normal_img_path)

In [ ]:
plt.imshow(img_Normal,cmap = 'gray')

In [ ]:
img_Normal.shape

In [ ]:
dim1 = []
dim2 = []
for image_filename in os.listdir(train_path_affected):
    
    img = plt.imread(train_path_affected+image_filename)
    d1 = img.shape[0]
    d2 = img.shape[1]
    dim1.append(d1)
    dim2.append(d2)

In [ ]:
sns.jointplot(dim1,dim2,alpha = .2)

In [ ]:
round(sum(dim1) / len(dim1))

In [ ]:
round(sum(dim2) / len(dim2))

In [ ]:
img_shape = (1200,800,1)

In [ ]:
img = Image.open(Normal_img_path)

In [ ]:
img

In [ ]:
new_image = img.resize(img_shape[:2])

In [ ]:
new_image.size

In [ ]:
img_arr = np.asarray(new_image)

In [ ]:
img_arr.shape

In [ ]:
plt.imshow(img_arr,cmap = 'gray')

In [ ]:
batch_size = 16

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(zoom_range=0.1,horizontal_flip=True)

In [ ]:
train_image_data = image_gen.flow_from_directory(train_path,
                                               target_size=img_shape[:2],
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
test_image_data = image_gen.flow_from_directory(test_path,
                                               target_size=img_shape[:2],
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(2,2),input_shape=img_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(2,2),input_shape=img_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(2,2),input_shape=img_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, remember its binary so we use sigmoid
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(patience = 2 , verbose = 1 , monitor='val_loss')

In [ ]:
results = model.fit_generator(train_image_data,epochs=20,
                              validation_data=test_image_data,
                             callbacks=[early_stop])

In [ ]:
model.save('model.h5')
